In [2]:
from datasets.dnerf_3d_video_IS import _load_data_from_json
from datasets.dnerf_3d_video_IS import dynerf_isg_weight
from datasets.dnerf_3d_video_IS import dynerf_ist_weight_nice
import os
import time
import torch

In [3]:
root_fp = '/home/loyot/workspace/SSD_1T/Datasets/NeRF/3d_vedio_datasets'
subject_id = 'flame_salmon_1'
split = 'train'
read_image = True
factor = 4
basedir = os.path.join(root_fp, subject_id)

In [3]:
(
    images,
    poses,
    timestamps,
    bounds,
    images_per_video,
    num_cameras,
    instrinc,
    render_poses,
    median_imgs,
) = _load_data_from_json(
    root_fp, 
    subject_id, 
    factor=factor, 
    split=split, 
    read_img=read_image,
    load_every=1 if split == "train" else 10,
)

poses shape:  (19, 3, 4)
loading video:


flame_salmon_1-cam01:   0%|          | 0/300 [00:00<?, ?it/s]

flame_salmon_1-cam20: 100%|██████████| 300/300 [00:02<00:00, 126.18it/s]


In [4]:
num_images = images.shape[0]
height = images.shape[1]
width = images.shape[2]
keyframes = False

In [5]:
print(height, width)

507 676


In [8]:
images.shape[1] * images.shape[2] 

342732

In [1]:
# Precompute ISG weights
t_s = time.time()
gamma = 1e-3 if keyframes else 2e-2
isg_weights = dynerf_isg_weight(
    images.view(-1, height, width, images.shape[-1]),
    median_imgs=median_imgs, gamma=gamma)
# Normalize into a probability distribution, to speed up sampling
isg_weights = (isg_weights.reshape(-1) / torch.sum(isg_weights))
torch.save(isg_weights, os.path.join(basedir, f"isg_weights.pt"))
t_e = time.time()
print(f"Computed {isg_weights.shape[0]} ISG weights in {t_e - t_s:.2f}s.")

NameError: name 'time' is not defined

In [7]:
# Precompute IST weights
t_s = time.time()
ist_weights = dynerf_ist_weight_nice(
    images.view(-1, height, width, images.shape[-1]),
    num_cameras=median_imgs.shape[0])
# Normalize into a probability distribution, to speed up sampling
ist_weights = (ist_weights.reshape(-1) / torch.sum(ist_weights))
torch.save(ist_weights, os.path.join(basedir, f"ist_weights.pt"))
t_e = time.time()
print(f"Computed {ist_weights.shape[0]} IST weights in {t_e - t_s:.2f}s.")

100%|██████████| 18/18 [05:08<00:00, 17.13s/it]


Computed 1850752800 IST weights in 315.41s.


In [8]:
ist_weights.shape

torch.Size([1850752800])

In [3]:
from datasets.dnerf_3d_video_IS import SubjectLoader

# training parameters
max_steps = 40000
init_batch_size = 2048
target_sample_batch_size = 1 << 18
weight_decay = 0.0
# scene parameters
aabb = torch.tensor([-1.0, -1.0, -1.0, 1.0, 1.0, 1.0])
near_plane = 0.2
far_plane = 1.0e10
# dataset parameters
train_dataset_kwargs = {"color_bkgd_aug": "random", "factor": 2}
test_dataset_kwargs = {"color_bkgd_aug": "black", "factor": 2}

grid_nlvl = 4
# model parameters
moving_step = 1/(2048*grid_nlvl)
hash_dst_resolution = 2048*grid_nlvl
grid_resolution = 128
# render parameters
render_step_size = 1e-3
alpha_thre = 1e-2
cone_angle = 0.004
milestones=[
    max_steps // 2,
    max_steps * 3 // 4,
    max_steps * 5 // 6,
    max_steps * 9 // 10,
]
lr = 1e-2

In [4]:
train_dataset = SubjectLoader(
    subject_id="flame_salmon_1",
    root_fp="/home/loyot/workspace/SSD_1T/Datasets/NeRF/3d_vedio_datasets",
    split="train",
    num_rays=init_batch_size,
    **train_dataset_kwargs,
)

poses shape:  (19, 3, 4)
loading video:


flame_salmon_1-cam01:   0%|          | 0/300 [00:00<?, ?it/s]

flame_salmon_1-cam20: 100%|██████████| 300/300 [00:09<00:00, 33.08it/s]


Reloaded 1850752800 ISG weights from file.
Reloaded 1850752800 IST weights from file.
render_poses:  torch.Size([300, 3, 4])
tensor([[-9.7229e-01,  1.0622e-03,  2.3377e-01, -4.2146e-01],
        [-1.0925e-03, -1.0000e+00,  1.8674e-18, -3.3667e-18],
        [-2.3377e-01,  2.5539e-04, -9.7229e-01,  1.5000e+00]])
showing a pose: 
[[ 0.8013   -0.03558  -0.597     0.728   ]
 [-0.05246  -0.9985   -0.010925  0.1616  ]
 [-0.5957    0.04007  -0.8022    1.323   ]]


In [15]:
for i in range(len(train_dataset)):
    train_dataset[i]